In [1]:
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt

In [2]:
data_path = "/home/yanjun/projects/dataset/kagglecatsanddogs_3367a/PetImages"
categories = ["Dog", "Cat"]

In [3]:
# 定义图像为50*50的大小
img_size = 50

train_data = []

def create_train_data():
    for category in categories:
        path = os.path.join(data_path, category)
        class_num = categories.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
                new_array = cv2.resize(img_array, (img_size, img_size))
                train_data.append([new_array, class_num])
            except Exception as e:
                pass

create_train_data()

In [4]:
print(len(train_data))

24946


In [5]:
import random

random.shuffle(train_data)

In [6]:
X =[]
y =[]
for feature, label in train_data:
    X.append(feature)
    y.append(label)
    
X = np.array(X).reshape(-1, img_size, img_size, 1) # 最后的1代表grayscale 只有1个通道

In [7]:
import pickle

# pickle保存数据
pickle_out = open("X.pickle", "wb")
pickle.dump(X, pickle_out)
pickle_out.close()

pickle_out = open("y.pickle", "wb")
pickle.dump(y, pickle_out)
pickle_out.close()

# pickle读取数据
#pickle_in = open("X.pickle", "rb")
#X = pickle.load(pickle_in)

In [8]:
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D

In [9]:
X = pickle.load(open("X.pickle","rb"))
y = pickle.load(open("y.pickle","rb"))

In [10]:
X = X / 255

In [11]:
import keras
model = keras.models.Sequential()

model.add(Conv2D(64, (3, 3), input_shape = X.shape[1:]))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten()) #Conv Layer是2D， DenseLayer是1D的 所以需要将ConvLayer压平
model.add(Dense(64))
model.add(Activation("relu"))

model.add(Dense(1))
model.add(Activation("sigmoid"))

model.compile(loss="binary_crossentropy",
             optimizer="adam",
             metrics=["accuracy"]) # 可以使用categorical_crossentropy作为损失函数

model.fit(X, y, batch_size =32, epochs=10, validation_split=0.1)

Using TensorFlow backend.


TypeError: The added layer must be an instance of class Layer. Found: <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f87c89f39b0>